In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WzdcVZiD5CcK4yTLocIJ")
project = rf.workspace("iit-jammu-fjbaf").project("human-yqjiq")
dataset = project.version(1).download("yolov8")
clear_output()

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
clear_output()

In [ ]:
# Run inference on an image with YOLOv8n
!yolo predict model=yolov8n.pt source='https://ultralytics.com/images/zidane.jpg'

In [ ]:
from ultralytics import YOLO

In [ ]:
def load_label(label_path):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            label_class, x1, y1, x2, y2 = line.split()
            x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            label_class = int(label_class)
            bboxes.append([x1, y1, x2, y2, label_class])

    return bboxes


In [ ]:
!pip install albumentations
clear_output()

In [ ]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

def load_label(label_path):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            label_class, x1, y1, x2, y2 = line.split()
            x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)
            label_class = int(label_class)
            bboxes.append([x1, y1, x2, y2, label_class])

    return bboxes

def apply_augmentations(image_paths, label_paths, output_dir, output_dir_l):
    os.makedirs(output_dir, exist_ok=True)

    for image_path, label_path in zip(image_paths, label_paths):
        img = Image.open(image_path)
        bboxes = load_label(label_path)

        augs = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
            A.Blur(blur_limit=(3, 7), p=0.1),
            A.RandomCrop(height=360, width=360, p=0.4),
            A.Sharpen(p=0.5)
        ],
        bbox_params=A.BboxParams(format="yolo"))

        img = np.array(img)
        augmented = augs(image=img, bboxes=bboxes)
        img, targets = augmented['image'], augmented['bboxes']

        # Convert targets to tensor
        targets = torch.as_tensor(targets, dtype=torch.float)

        # Save the augmented image and targets
        image_name = os.path.basename(image_path)
        torch.save(img, os.path.join(output_dir, f"augmented_{image_name}"))
        with open(os.path.join(output_dir_l, f"augmented_{image_name}.txt"), 'w') as f:
            for bbox in targets:
                x1, y1, x2, y2, label_class = bbox
                f.write(f"{label_class} {x1} {y1} {x2} {y2}\n")

def merge_data(original_image_paths, augmented_image_paths, original_label_paths, augmented_label_paths, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for orig_img_path, aug_img_path, orig_label_path, aug_label_path in zip(
            original_image_paths, augmented_image_paths, original_label_paths, augmented_label_paths):

        # Read original and augmented images
        orig_img = cv2.imread(orig_img_path)
        aug_img = cv2.imread(aug_img_path)

        # Ensure images have the same dimensions
        min_height = min(orig_img.shape[0], aug_img.shape[0])
        min_width = min(orig_img.shape[1], aug_img.shape[1])

        orig_img = orig_img[:min_height, :min_width, :]
        aug_img = aug_img[:min_height, :min_width, :]

        # Read original and augmented labels
        with open(orig_label_path, 'r') as orig_label_file:
            orig_labels = orig_label_file.readlines()

        with open(aug_label_path, 'r') as aug_label_file:
            aug_labels = aug_label_file.readlines()

        # Combine original and augmented labels
        combined_labels = orig_labels + aug_labels

        # Save merged image
        merged_img = np.concatenate((orig_img, aug_img), axis=1)
        merged_img_path = os.path.join(output_dir, f"merged_{os.path.basename(orig_img_path)}")
        cv2.imwrite(merged_img_path, merged_img)

        # Save merged labels
        merged_label_path = os.path.join(output_dir, f"merged_{os.path.basename(orig_label_path)}")
        with open(merged_label_path, 'w') as merged_label_file:
            merged_label_file.writelines(combined_labels)

# Example usage:
original_image_paths = [f'/content/Human-1/train/images/{filename}' for filename in os.listdir('/content/Human-1/train/images/')]
original_label_paths = [f'/content/Human-1/train/labels/{filename}' for filename in os.listdir('/content/Human-1/train/labels/')]

augmented_image_paths = [f'/content/temp/{filename}' for filename in os.listdir('/content/temp/') if filename.startswith('augmented_')]
augmented_label_paths = [f'/content/temp/{filename}' for filename in os.listdir('/content/temp/') if filename.startswith('augmented_')]

output_directory = '/content/merged/'

# Apply augmentations
apply_augmentations(original_image_paths, original_label_paths, '/content/temp/', '/content/temp/')

# Merge original and augmented data
merge_data(original_image_paths, augmented_image_paths, original_label_paths, augmented_label_paths, output_directory)

# Example usage:
image_paths = [f'/content/Human-1/train/images/{filename}' for filename in os.listdir('/content/Human-1/train/images/')]
label_paths = [f'/content/Human-1/train/labels/{filename}' for filename in os.listdir('/content/Human-1/train/labels/')]
output_directory = '/content/temp/images'
output_directory_l = '/content/temp/labels'


apply_augmentations(image_paths, label_paths, output_directory,output_directory_l)


In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/train4/weights/last.pt data=/content/Human-1/data.yaml conf = 0.7

In [ ]:
import locale
print(locale.getpreferredencoding())
print()
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8

UTF-8


In [ ]:
model = YOLO('yolov8n.pt')
# results = model.train(data='data.yaml', epochs=20)

In [ ]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [ ]:
result = model.predict(source = "/content/WIN_20231117_16_40_07_Pro.mp4", save = True)